In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 0 #当日は0、前日は1・・・
date2 = "08/19"
pmc = "15022008" # 店舗ID
SPREADSHEET_KEY = '1sbmnCWqVuVWvT33aJECgEuJTU88c--XHjWB0P1ikzlc' # スプレッドシートのID
pstart = 1 #1 パチンコの1台目
sstart = 851 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"


Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
chrome_service = ChromeService(executable_path='C:\\Users\\garden026\\WD\\114.0.5735.90\\chromedriver.exe')

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "112":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "112":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})

['08/19', '025812', '1', '7436.0', '1506.0', '5', '1']
['08/19', '025794', '2', '21026.0', '6019.0', '16', '4']
['08/19', '025830', '3', '9487.0', '-7522.0', '0', '0']
['08/19', '025223', '5', '17692.0', '-8726.0', '6', '5']
['08/19', '025544', '6', '8974.0', '8126.0', '48', '3']
['08/19', '025740', '7', '20769.0', '-12939.0', '5', '2']
['08/19', '025509', '8', '17179.0', '3010.0', '22', '9']
['08/19', '025780', '10', '19487.0', '6019.0', '43', '12']
['08/19', '025712', '11', '16667.0', '-1805.0', '40', '6']
['08/19', '025473', '12', '18205.0', '2408.0', '18', '7']
['08/19', '025392', '13', '31282.0', '-18055.0', '18', '12']
['08/19', '025796', '15', '6410.0', '18658.0', '16', '1']
['08/19', '025839', '16', '21538.0', '-12036.0', '6', '2']
['08/19', '025724', '17', '3846.0', '7825.0', '22', '1']
['08/19', '025764', '18', '37692.0', '-21666.0', '6', '4']
['08/19', '025606', '20', '32308.0', '5117.0', '25', '9']
['08/19', '025606', '21', '15897.0', '-1805.0', '9', '4']
['08/19', '025606'

['08/19', '025699', '217', '22564.0', '10533.0', '40', '9']
['08/19', '025735', '218', '12564.0', '3010.0', '16', '8']
['08/19', '025826', '220', '29231.0', '-20763.0', '2', '2']
['08/19', '025579', '221', '25385.0', '-12638.0', '5', '2']
['08/19', '025579', '222', '20513.0', '-7221.0', '5', '2']
['08/19', '025579', '223', '20513.0', '5719.0', '14', '4']
['08/19', '025579', '225', '12308.0', '12941.0', '12', '3']
['08/19', '025579', '226', '29744.0', '6621.0', '16', '4']
['08/19', '025579', '227', '16154.0', '18658.0', '21', '3']
['08/19', '025579', '228', '25128.0', '14746.0', '21', '5']
['08/19', '025579', '230', '10769.0', '1806.0', '6', '2']
['08/19', '025579', '231', '19744.0', '7223.0', '13', '3']
['08/19', '025579', '232', '24872.0', '5418.0', '16', '4']
['08/19', '025579', '233', '18462.0', '10533.0', '16', '3']
['08/19', '025579', '235', '23846.0', '-2106.0', '9', '4']
['08/19', '025579', '236', '16667.0', '9931.0', '14', '2']
['08/19', '025579', '237', '44103.0', '-24073.0', 

['08/19', '025869', '535', '0', '0', '0', '0']
['08/19', '025869', '536', '0', '0', '0', '0']
['08/19', '025869', '537', '0', '0', '0', '0']
['08/19', '025306', '538', '0', '0', '0', '0']
['08/19', '025750', '550', '5385.0', '15649.0', '51', '3']
['08/19', '025656', '551', '8718.0', '5117.0', '14', '6']
['08/19', '025362', '552', '769.0', '2709.0', '3', '1']
['08/19', '025446', '553', '0', '0', '0', '0']
['08/19', '025208', '555', '0', '0', '0', '0']
['08/19', '025775', '556', '4872.0', '-1203.0', '3', '2']
['08/19', '025795', '557', '769.0', '3010.0', '4', '1']
['08/19', '025829', '558', '0', '0', '0', '0']
['08/19', '025800', '560', '769.0', '-1203.0', '0', '0']
['08/19', '025837', '561', '27949.0', '1506.0', '15', '3']
['08/19', '025766', '562', '0', '0', '0', '0']
['08/19', '025850', '563', '513.0', '-1203.0', '0', '0']
['08/19', '025852', '565', '0', '0', '0', '0']
['08/19', '025852', '566', '0', '0', '0', '0']
['08/19', '025856', '567', '17692.0', '-902.0', '10', '3']
['08/19', '

['08/19', '120096', '855', '452', '-139.0', '2', '0', '--']
['08/19', '120141', '856', '4228', '2315.0', '25', '11', '--']
['08/19', '120141', '857', '2380', '-324.0', '9', '6', '--']
['08/19', '119916', '858', '4984', '-139.0', '18', '13', '--']
['08/19', '119916', '860', '700', '-278.0', '1', '2', '--']
['08/19', '119916', '861', '1086', '-185.0', '3', '3', '--']
['08/19', '119916', '862', '1506', '-417.0', '4', '5', '--']
['08/19', '119916', '863', '2118', '-139.0', '8', '3', '--']
['08/19', '119996', '865', '4544', '139.0', '19', '9', '--']
['08/19', '119996', '866', '5485', '417.0', '23', '13', '--']
['08/19', '119996', '867', '1878', '-648.0', '5', '4', '--']
['08/19', '119996', '868', '2622', '-833.0', '8', '5', '--']
['08/19', '120010', '870', '2090', '-1574.0', '2', '4', '--']
['08/19', '120010', '871', '968', '-417.0', '3', '0', '--']
['08/19', '120010', '872', '2398', '139.0', '9', '5', '--']
['08/19', '120010', '873', '3884', '-324.0', '16', '6', '--']
['08/19', '120010', '

['08/19', '120110', '1166', '2468', '-278.0', '13', '8', '--']
['08/19', '120110', '1167', '1714', '1481.0', '19', '3', '--']
['08/19', '120110', '1168', '3384', '-1944.0', '10', '5', '--']
['08/19', '120110', '1170', '3044', '1389.0', '25', '10', '--']
['08/19', '120110', '1171', '4637', '-1111.0', '21', '20', '--']
['08/19', '120021', '1172', '1648', '-324.0', '8', '10', '--']
['08/19', '120021', '1173', '0', '0', '0', '0', '--']
['08/19', '120021', '1175', '890', '-787.0', '2', '3', '--']
['08/19', '120021', '1176', '0', '0', '0', '0', '--']
['08/19', '120021', '1177', '30', '0', '0', '0', '--']
['08/19', '120021', '1178', '981', '-185.0', '6', '1', '--']
['08/19', '120021', '1180', '99', '833.0', '4', '2', '--']
['08/19', '120021', '1181', '34', '0', '0', '0', '--']
['08/19', '120021', '1182', '728', '880.0', '9', '1', '--']
['08/19', '120068', '1183', '101', '0', '0', '0', '--']
['08/19', '120068', '1185', '522', '1435.0', '8', '7', '--']
['08/19', '120068', '1186', '108', '-139.0

{'spreadsheetId': '1sbmnCWqVuVWvT33aJECgEuJTU88c--XHjWB0P1ikzlc',
 'updates': {'spreadsheetId': '1sbmnCWqVuVWvT33aJECgEuJTU88c--XHjWB0P1ikzlc',
  'updatedRange': 'S20RAW!F27303:M27462',
  'updatedRows': 160,
  'updatedColumns': 8,
  'updatedCells': 1280}}